Basic Text Pre-processing and Topic Modelling
======

In [1]:
import glob
import math
import re
import threading
import time

import gensim
import nltk
import pyLDAvis
import tomotopy as tp
import tqdm

Data ingestion
-----

In [2]:
data_files = glob.glob("./data/bbc/*/*.txt")

In [3]:
raw_docs = []
for file in tqdm.tqdm(data_files):
    with open(file) as f:
        doc = f.read()
        raw_docs.append(doc)

print(raw_docs[0])

100%|██████████| 2225/2225 [00:00<00:00, 12015.29it/s]

Ad sales boost Time Warner profit

Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.

The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.

Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AO

Text pre-processing and tokenisation
------

- Naive tokenisation (by whitespace)
- Strip leading/trailing non-informative punctuation from tokens

In [4]:
remove_punctuation = "'\"()[]?!,.:;<>/|_"


def naive_tokenise(doc):
    tokens = doc.split()
    tokens = [x.strip(remove_punctuation) for x in tokens]
    return tokens


docs = [naive_tokenise(doc) for doc in raw_docs]

print(" ".join(docs[0]))

Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn Â£600m for the three months to December from $639m year-earlier The firm which is now one of the biggest investors in Google benefited from sales of high-speed internet connections and higher advert sales TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros and less users for AOL Time Warner said on Friday that it now owns 8% of search-engine Google But its own internet business AOL had has mixed fortunes It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters However the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing custome

Chunk into significant bigrams/trigrams based on collocation frequency

- Min count: Must appear in at least 0.1% of the documents

- Scoring: "default" or "npmi"

- Threshold: Intuitively, higher threshold means fewer phrases. With the default scorer, this is greater than or equal to 0; with the NPMI scorer, this is in the range -1 to 1.

- Common terms: These terms will be ignored if they come between normal words. E.g., if `common_terms` includes the word "of", then when the phraser sees "Wheel of Fortune" it actually evaluates _"Wheel Fortune"_ as an n-gram, putting "of" back in only at the output level.

In [5]:
min_count = math.ceil(len(docs) / 1000)
scoring = "npmi"
# We want a relatively high threshold so that we don't start littering spurious n-grams all over our corpus, diluting our results.
# E.g., we want "Lord_of_the_Rings", but not "slightly_better_than_analysts"
threshold = 0.75
common_terms = ["a", "an", "the", "of", "on", "in", "at"]

This could take a while, so set up a threaded function with a basic progress indicator in the main thread

In [6]:
def find_ngrams(docs, results):
    bigram = gensim.models.Phrases(
        docs,
        min_count=min_count,
        threshold=threshold,
        scoring=scoring,
        common_terms=common_terms,
    )
    trigram = gensim.models.Phrases(
        bigram[docs],
        min_count=min_count,
        threshold=threshold,
        scoring=scoring,
        common_terms=common_terms,
    )

    # Finalise the bigram/trigram generators for efficiency
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    results[0] = bigram_mod
    results[1] = trigram_mod

In [7]:
print("Generating n-grams", flush=True, end="")

results = [None, None]
t = threading.Thread(target=find_ngrams, args=(docs, results))
t.start()

progress_countdown = 1.0

while t.isAlive():
    time.sleep(0.1)
    progress_countdown -= 0.1
    if progress_countdown <= 0:
        print(" .", flush=True, end="")
        progress_countdown = 1

print(" Done.")

bigram_mod = results[0]
trigram_mod = results[1]

docs = [trigram_mod[bigram_mod[doc]] for doc in docs]

print(" ".join(docs[0]))

Generating n-grams . . . . . . . . . . Done.
Ad sales boost Time Warner profit Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn Â£600m for the three months to December from $639m year-earlier The firm which is now one of the biggest investors in Google benefited from sales of high-speed internet connections and higher advert sales TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn Its profits were buoyed by one-off gains which offset a profit dip at Warner_Bros and less users for AOL Time Warner said on Friday that it now owns 8% of search-engine Google But its own internet business AOL had has mixed fortunes It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters However the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues It hopes to increase subscribers by offering the online service free to TimeWarner internet customers a

Second-pass tokenisation
- Case folding
- Remove single apostrophes
- Remove stop words, remove purely numeric/non-alphabetic tokens

In [8]:
# stopset = set(nltk.corpus.stopwords.words("english"))
# Testing term weighting
stopset = []


def second_tokenise(tokens):
    new_tokens = []
    for token in tokens:
        token = token.casefold()
        token = token.replace("'", "")
        if token in stopset or re.match("^[^a-z]+$", token):
            continue
        new_tokens.append(token)

    return new_tokens


docs = [second_tokenise(doc) for doc in docs]

print(" ".join(docs[0]))

ad sales boost time warner profit quarterly profits at us media giant timewarner jumped to $1.13bn â£600m for the three months to december from $639m year-earlier the firm which is now one of the biggest investors in google benefited from sales of high-speed internet connections and higher advert sales timewarner said fourth quarter sales rose to $11.1bn from $10.9bn its profits were buoyed by one-off gains which offset a profit dip at warner_bros and less users for aol time warner said on friday that it now owns of search-engine google but its own internet business aol had has mixed fortunes it lost subscribers in the fourth quarter profits were lower than in the preceding three quarters however the company said aols underlying profit before exceptional items rose on the back of stronger internet advertising revenues it hopes to increase subscribers by offering the online service free to timewarner internet customers and will try to sign up aols existing customers for high-speed broad

Model training (LDA)
----

Add processed docs to the LDA model and train it.

The random seed and parallelisation can affect results, so setting both the seed and number of workers is necessary for reproducibility.

In [9]:
# Persistence
model_seed = 11399
num_workers = 12

# Model options
model_file = "model.bin"
num_topics = 20

# Training iterations
load_saved_model = False
burn_in = 500
train_batch = 100
train_total = 1000

# Extended training
train_until_min_ll = False
max_iterations = 10000

In [10]:
if load_saved_model:
    model = tp.LDAModel.load(model_file)
    print(f"Loaded from '{model_file}'.")
else:
    model = tp.LDAModel(tw=tp.TermWeight.IDF, seed=model_seed, k=num_topics)
    
    model.burn_in = burn_in

    for doc in tqdm.tqdm(docs):
        model.add_doc(doc)

    model.train(0, workers=num_workers, parallel=tp.ParallelScheme.DEFAULT)
    print(
        f"Num docs: {len(model.docs)}, Vocab size: {model.num_vocabs}, "
        f"Num words: {model.num_words}"
    )
    print(f"Removed top words: {model.removed_top_words}")

    print("Training model...", flush=True)

    try:
        for i in range(0, train_total, train_batch):
            start_time = time.perf_counter()
            model.train(
                train_batch, workers=num_workers, parallel=tp.ParallelScheme.DEFAULT
            )
            elapsed = time.perf_counter() - start_time
            print(
                f"Iteration: {i + train_batch}\tLog-likelihood: {model.ll_per_word}\t"
                f"Time: {elapsed:.3f}s",
                flush=True,
            )
    except KeyboardInterrupt:
        print("Stopping train sequence.")
    model.save(model_file)
    print(f"Saved to '{model_file}'.")

100%|██████████| 2225/2225 [00:00<00:00, 15990.99it/s]

Num docs: 2225, Vocab size: 34869, Num words: 823429
Removed top words: []
Training model...


Iteration: 100	Log-likelihood: -20.73482849172073	Time: 1.229s
Iteration: 200	Log-likelihood: -20.440579817304194	Time: 1.204s
Iteration: 300	Log-likelihood: -20.30484707503074	Time: 1.202s
Iteration: 400	Log-likelihood: -20.198855351873114	Time: 1.227s
Iteration: 500	Log-likelihood: -20.126639099195934	Time: 1.201s
Iteration: 600	Log-likelihood: -20.041293780078465	Time: 1.441s
Iteration: 700	Log-likelihood: -19.987318535794316	Time: 1.340s
Iteration: 800	Log-likelihood: -19.941006512922034	Time: 1.333s
Iteration: 900	Log-likelihood: -19.898092991563875	Time: 1.339s
Iteration: 1000	Log-likelihood: -19.870536043329185	Time: 1.331s
Saved to 'model.bin'.


In [11]:
if train_until_min_ll:
    print("Continuing to train until minimum log-likelihood...")
    print("(N.B.: This may not correlate with increased human interpretability)")
    last_ll = model.ll_per_word
    i = 0
    consecutive_loss = 0

    while True:
        try:
            start_time = time.perf_counter()
            model.train(
                train_batch, workers=num_workers, parallel=tp.ParallelScheme.DEFAULT
            )
            i += train_batch
            elapsed = time.perf_counter() - start_time
            print(
                f"Iteration: {i}\tLog-likelihood: {model.ll_per_word}\t"
                f"Time: {elapsed:.3f}s",
                flush=True,
            )

            if model.ll_per_word < last_ll:
                consecutive_loss += 1
            else:
                consecutive_loss = 0
                model.save(model_file)
            last_ll = model.ll_per_word

            if consecutive_loss == 2 or i >= max_iterations:
                break

        except KeyboardInterrupt:
            print("Stopping extended train sequence.")
            break

    model = tp.LDAModel.load(model_file)
    print(f"Best recent model saved at '{model_file}' (LL: {model.ll_per_word}).")

Continuing to train until minimum log-likelihood...
(N.B.: This may not correlate with increased human interpretability)
Iteration: 100	Log-likelihood: -19.854442542459015	Time: 1.362s
Iteration: 200	Log-likelihood: -19.839756952900004	Time: 1.367s
Iteration: 300	Log-likelihood: -19.825716858055284	Time: 1.336s
Iteration: 400	Log-likelihood: -19.8176962347457	Time: 1.342s
Iteration: 500	Log-likelihood: -19.80614527892285	Time: 1.346s
Iteration: 600	Log-likelihood: -19.80007906914145	Time: 1.341s
Iteration: 700	Log-likelihood: -19.787514303434968	Time: 1.347s
Iteration: 800	Log-likelihood: -19.77786234644422	Time: 1.340s
Iteration: 900	Log-likelihood: -19.775727325337794	Time: 1.351s
Iteration: 1000	Log-likelihood: -19.76928624098216	Time: 1.353s
Iteration: 1100	Log-likelihood: -19.767355681049164	Time: 1.336s
Iteration: 1200	Log-likelihood: -19.762648512313486	Time: 1.327s
Iteration: 1300	Log-likelihood: -19.759945126423847	Time: 1.355s
Iteration: 1400	Log-likelihood: -19.7590555323462

Topic labelling

In [ ]:
print("Extracting suggested topic labels...", flush=True)
# extractor = tp.label.PMIExtractor(min_cf=10, min_df=5, max_len=5, max_cand=10000)
extractor = tp.label.PMIExtractor(min_cf=5, min_df=3, max_len=5, max_cand=20000)
candidates = extractor.extract(model)
# labeler = tp.label.FoRelevance(model, candidates, min_df=5, smoothing=1e-2,
# mu=0.25)
labeler = tp.label.FoRelevance(
    model, candidates, min_df=3, smoothing=1e-2, mu=0.25, workers=num_workers
)
print("Done.")

In [13]:
Print results
------

Extracting suggested topic labels...
Done.


Print results
------

In [13]:
def print_topic(topic_id):
    # Labels
    labels = ", ".join(
        label for label, score in labeler.get_topic_labels(topic_id, top_n=10)
    )
    print(f"Suggested labels: {labels}")

    # Print this topic
    words_probs = model.get_topic_words(topic_id, top_n=10)
    words = [x[0] for x in words_probs]

    words = ", ".join(words)
    print(words)

In [14]:
for k in range(model.k):
    print(f"[Topic {k+1}]")
    print_topic(k)
    print()

[Topic 1]
Suggested labels: chrysler, high fuel prices, full-year, profits, profits fall, new planes, operating profits, profit, high fuel, full year
sales, profits, euros, shares, airline, airlines, profit, company, boeing, firm

[Topic 2]
Suggested labels: steve morgan, if gerrard, steven is, stalking_horse, we are also, have a common, liverpool with, walk out of, money and it, new investment
liverpool, gerrard, ebbers, parry, mci, steven, worldcom, sullivan, apple, club

[Topic 3]
Suggested labels: users, web, software, websites, program, viruses, malicious, anti-virus, programs, windows
software, users, search, security, sites, microsoft, e-mail, site, information, spam

[Topic 4]
Suggested labels: modern day, surely, this is just, etc, creatures, sadly, but sadly, up your, cgi, you may
ukip, kilroy-silk, blogs, blog, she, you, your, her, veritas, radio

[Topic 5]
Suggested labels: years later he, well done, modern day, malcolm_x, stone_roses, joy_division, harlem, first novel, ala

Visualise
--------
- Present data in the format expected by pyLDAvis

In [15]:
model_data = {
    "topic_term_dists": [model.get_topic_word_dist(k) for k in range(model.k)],
    "doc_topic_dists": [model.docs[n].get_topic_dist() for n in range(len(model.docs))],
    "doc_lengths": [len(model.docs[n].words) for n in range(len(model.docs))],
    "vocab": model.vocabs,
    "term_frequency": model.vocab_freq,
}

Again, this could take a while

In [16]:
def prepare_vis(model_data, results):
    vis_data = pyLDAvis.prepare(**model_data)
    results[0] = vis_data

In [17]:
print("Preparing LDA visualisation", flush=True, end="")

results = [None]
t = threading.Thread(target=prepare_vis, args=(model_data, results))
t.start()

progress_countdown = 1.0

while t.isAlive():
    time.sleep(0.1)
    progress_countdown -= 0.1
    if progress_countdown <= 0:
        print(" .", flush=True, end="")
        progress_countdown = 1

print(" Done.")

vis_data = results[0]

Preparing LDA visualisation . . . . . . . . . . . . . . . . . . Done.


In [18]:
pyLDAvis.display(vis_data)

Iterate
--------
- See what the main topics might be, slice initial corpus and re-run LDA to get sub-topics